In [1]:
import os
import geopandas as gpd
import pandas as pd

# === FOLDERS ===
folders = [
    r"C:\Users\Admin\OneDrive - United States International University (USIU)\Desktop\USIU MAp Project\Raw\Pedistrain Way",
    r"C:\Users\Admin\OneDrive - United States International University (USIU)\Desktop\USIU MAp Project\Raw\Car way"
]
output_file = r"C:\Users\Admin\OneDrive - United States International University (USIU)\Desktop\USIU MAp Project\data\combined_cleaned_routes.geojson"

# === CORRECTIONS ===
corrections = {
    "Adminstartion Block": "Administration Block",
    "Adminstartion Block Parking Lot": "Administration Block Parking Lot",
    "Adminstartion Stting Area": "Administration Sitting Area",
    "Audoitoruim": "Auditorium",
    "Auditoruim": "Auditorium",
    "LIlian Beam": "Lilian Beam (ICT Centere)",
    "Lilian Beam(ICT Centere)": "Lilian Beam (ICT Centere)",
    "Lilian Beam Building(ICT Cenetre)": "Lilian Beam (ICT Centere)",
    "Lilian Beam Building( ICT Centere)": "Lilian Beam (ICT Centere)",
    "Freida Brown  Student Centere Parking Lot": "Frieda Brown Student Centere Parking Lot",
    "Swiming Pool": "Swimming Pool",
    "Pual's Caffe": "Paul's Caffe",
    "Scince Complex": "Science Complex Block",
    "School of Graudate Studies": "School of Graduate Studies",
    "School of School of Business": "School of Business Block",
    "I&J Lecture Halls": "I and J Lecture Halls",
    "E & F Lecture Halls": "E and F Lecture Halls",
    "School of Humanities and Social Sciences parking Lot": "School of Humanities and Social Sciences Parking Lot",
    "to Sewerage Plant": "Sewerage Plant",
    "Adminstration Block":"Administration Block",
    "Adminstration Block Parking plot":"Administration Block Parking Lot",
    "Swiming Pool Parking Lot":"Swimming Pool Parking Lot",
    "Swimming Pool Parking":"Swimming Pool Parking Lot",
    "Sewerage Plan":"Sewerage Plant",
    "Swarage Plant":"Sewerage Plant",
    "Science Complex":"Science Complex Block",
    "Science Complex Parking Plot":"Science Complex Parking Lot",
    "School of Business":"School of Business Block",
    "School of Humanities":"School of Humanities and Social Sciences Block",
    "School of Humanities Parking Plot":"School of Humanities and Social Sciences Parking Lot"
}

geojsons = []

# === PROCESS ALL FILES IN FOLDERS ===
for folder_path in folders:
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".json") or file_name.endswith(".geojson"):
            file_path = os.path.join(folder_path, file_name)

            try:
                gdf = gpd.read_file(file_path)
            except Exception as e:
                print(f"⚠️ Could not read {file_name}: {e}")
                continue

            if gdf.empty or gdf.geometry.is_empty.any():
                print(f"⚠️ Skipping {file_name}: Empty geometry.")
                continue

            if not all(g.geom_type == "LineString" for g in gdf.geometry):
                print(f"⚠️ Skipping {file_name}: Not all geometries are LineStrings.")
                continue

            # Route name from filename
            route_name = file_name.replace(".json", "").replace(".geojson", "")
            if " to " in route_name:
                start_name, end_name = route_name.split(" to ", 1)
            else:
                parts = route_name.split()
                to_index = parts.index("to") if "to" in parts else -1
                start_name = " ".join(parts[:to_index]) if to_index != -1 else route_name
                end_name = " ".join(parts[to_index + 1:]) if to_index != -1 else "Unknown"

            start_name = corrections.get(start_name.strip(), start_name.strip())
            end_name = corrections.get(end_name.strip(), end_name.strip())

            # Get coordinates
            coords = list(gdf.geometry.iloc[0].coords)
            start_coord = coords[0]
            end_coord = coords[-1]

            # Add metadata
            gdf["route_name"] = route_name
            gdf["start_name"] = start_name
            gdf["end_name"] = end_name
            gdf["start_coord"] = [start_coord] * len(gdf)
            gdf["end_coord"] = [end_coord] * len(gdf)

            geojsons.append(gdf)

# === EXPORT COMBINED DATA ===
if geojsons:
    combined_gdf = gpd.GeoDataFrame(pd.concat(geojsons, ignore_index=True), crs=geojsons[0].crs)
    combined_gdf.to_file(output_file, driver="GeoJSON")
    print(f"\n✅ Combined and cleaned GeoJSON saved to:\n{output_file}")
else:
    print("❌ No valid GeoJSONs found.")



✅ Combined and cleaned GeoJSON saved to:
C:\Users\Admin\OneDrive - United States International University (USIU)\Desktop\USIU MAp Project\data\combined_cleaned_routes.geojson
